<a href="https://colab.research.google.com/github/Chanezig/Associations/blob/main/Association_APIORI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5953 sha256=d38e4e77b22b2db0b6c9e153e9afa05ff35323be65a0dd7c4156b8d94f390b70
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori


In [ ]:
import matplotlib.pyplot as plt
from apyori import apriori

In [ ]:
# Open the dataset and convert to list
transactions = []

with open('/content/groceries100 (3).csv') as f:
    next(f) # Skip the first line
    for line in f:
        transaction = [item for item in line.strip().split(',') if item != '']
        transactions.append(transaction)
# Delete elements that originate from the first column - Item(s)
for lists in transactions:
    del lists[0]
transactions[:5]

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product']]

In [ ]:
rules = list(apriori(
    transactions,
    min_support=0.001,
    min_lift =1.5,
    min_confidence=0.10,
    min_length=2,
    max_length=2 ))
# Print one rule
print(rules[0])

RelationRecord(items=frozenset({'bottled beer', 'UHT-milk'}), support=0.010101010101010102, ordered_statistics=[OrderedStatistic(items_base=frozenset({'UHT-milk'}), items_add=frozenset({'bottled beer'}), confidence=0.5, lift=16.5), OrderedStatistic(items_base=frozenset({'bottled beer'}), items_add=frozenset({'UHT-milk'}), confidence=0.33333333333333337, lift=16.5)])


In [ ]:
rules_df = pd.DataFrame(
    [{'From': list(rule[0])[0],
      'To': list(rule[0])[1],
      'Support': rule[1],
      'Confidence': rule[2][0][2],
      'Lift': rule[2][0][3]} for rule in rules if len(rule[0]) == 2])
# Remove all rows with missing values
rules_df = rules_df.dropna()
rules_df.head()

,From,To,Support,Confidence,Lift
0,bottled beer,UHT-milk,0.010101,0.5,16.500
1,bottled water,UHT-milk,0.010101,0.5,4.125
2,butter,UHT-milk,0.010101,0.5,8.250
3,curd,UHT-milk,0.010101,0.5,8.250
4,hard cheese,UHT-milk,0.010101,0.5,24.750


In [ ]:
# Pick top rules
rules_df = rules_df.sort_values('Support', ascending=False).head(50)

# List of all items
items = set(rules_df['From']) | set(rules_df['To'])

# Creates a mapping of items to numbers
imap = {item: i for i, item in enumerate(items)} # Adds numbers to items - enumerate()

# Maps the items to numbers and adds the nummeric 'FromN' and 'ToN' columns
rules_df['FromN'] = rules_df['From'].map(imap)
rules_df['ToN'] = rules_df['To'].map(imap)

# Displays the top 20 association rules, sorted by Support
rules_df.head(20)

,From,To,Support,Confidence,Lift,FromN,ToN
465,whole milk,other vegetables,0.070707,0.411765,1.630588,19,0
555,whole milk,yogurt,0.060606,0.400000,1.584000,19,24
454,root vegetables,other vegetables,0.050505,0.555556,3.235294,13,0
501,rolls/buns,sausage,0.050505,0.625000,2.946429,20,11
504,soda,rolls/buns,0.050505,0.384615,1.813187,2,20
284,curd,whole milk,0.040404,0.666667,2.640000,18,19
95,bottled water,rolls/buns,0.040404,0.333333,1.571429,28,20
460,tropical fruit,other vegetables,0.040404,0.363636,2.117647,10,0
499,root vegetables,rolls/buns,0.040404,0.444444,2.095238,13,20
538,soda,whole milk,0.040404,0.307692,1.218462,2,19
